In [1]:
!pip install pandas
!pip install opencv-python
!apt-get install -y libgl1
!pip install kaggle


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1 is already the newest version (1.4.0-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import h5py
import shutil
import cv2
import random
import uuid

2024-08-23 04:24:31.248826: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-23 04:24:31.283195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-23 04:24:31.293556: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-23 04:24:31.376215: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

def add_noise(img):
    gauss_noise=np.zeros(img.shape,dtype=np.uint8)
    cv2.randn(gauss_noise,128,20)
    gauss_noise=(gauss_noise*0.5).astype(np.uint8)
    gn_img=cv2.add(img,gauss_noise)
    return gn_img

def random_rotate(img,low=-40,high=40):
    angle = np.random.randint(low,high)
    rows,cols = img.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle ,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    return dst

def vertical_flip(img):
    return cv2.flip(img, 1)

def horizontal_flip(img):
    return cv2.flip(img,0)

def horizontal_vertical_flip(img):
    return cv2.flip(img,-1)

def horizontal_shearing(img):
    M = np.float32([[1, np.random.uniform(), 0], [0, 1, 0], [0, 0, 1]])
    rows,cols = img.shape[:2]
    sheared_img = cv2.warpPerspective(img, M, (int(cols*1.5), int(rows*1.5)))
    return sheared_img

def vertical_shearing(img):
    rows,cols = img.shape[:2]
    M = np.float32([[1, 0, 0], [np.random.uniform(), 1, 0], [0, 0, 1]])
    sheared_img = cv2.warpPerspective(img, M,
                                 (int(cols*1.5),
                                 int(rows*1.5)))
    return sheared_img

def horizontal_vertical_shearing(img):
    rows, cols = img.shape[:2]
    M = np.float32([[1, np.random.uniform(), 0], [np.random.uniform(), 1, 0], [0, 0, 1]])
    sheared_img = cv2.warpPerspective(img, M,
                                 (int(cols*1.5),
                                 int(rows*1.5)))
    return sheared_img
def salt_and_pepper_noise(img):

    # Getting the dimensions of the image
    row , col = img.shape[:2]

    # Randomly pick some pixels in the
    # image for coloring them white
    # Pick a random number between 300 and 10000
    number_of_pixels = np.random.randint(200,10000)
    for i in range(number_of_pixels):

        # Pick a random y coordinate
        y_coord=np.random.randint(0, row - 1)

        # Pick a random x coordinate
        x_coord=np.random.randint(0, col - 1)

        # Color that pixel to white
        img[y_coord][x_coord] = 255

    # Randomly pick some pixels in
    # the image for coloring them black
    # Pick a random number between 300 and 10000
    number_of_pixels = np.random.randint(50,200)
    for i in range(number_of_pixels):

        # Pick a random y coordinate
        y_coord=np.random.randint(0, row - 1)

        # Pick a random x coordinate
        x_coord=np.random.randint(0, col - 1)

        # Color that pixel to black
        img[y_coord][x_coord] = 0

    return img

def random_brightness_contrast(image, brightness_range=(-0.5, 0.5), contrast_range=(-0.5, 0.5)):
  """
  Applies random brightness and contrast to an image.

  Args:
    image: The input image.
    brightness_range: A tuple specifying the range of brightness adjustment.
    contrast_range: A tuple specifying the range of contrast adjustment.

  Returns:
    The adjusted image.
  """

  # Convert the image to floating-point representation for numerical operations
  image = image.astype(np.float32) / 255.0

  # Generate random brightness and contrast values within the specified ranges
  brightness = np.random.uniform(brightness_range[0], brightness_range[1])
  contrast = np.random.uniform(contrast_range[0], contrast_range[1])
  # Adjust brightness
  image = image + brightness

  # Adjust contrast
  image = (image - 0.5) * contrast + 0.5

  # Clip the values to the range [0, 1]
  image = np.clip(image, 0, 1)

  # Convert the image back to integer representation (0-255)
  image = (image * 255).astype(np.uint8)

  return image

def grayscale_3channels(image):
  """
  Converts an image to grayscale while preserving 3 channels.

  Args:
    image: The input image.

  Returns:
    The grayscale image with 3 channels.
  """

  # Convert the image to grayscale using the average method
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Repeat the grayscale channel to create 3 channels
  gray_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)

  return gray_image

def poisson_noise(image):
    noisy_image = np.random.poisson(image)
    noisy_image = np.clip(noisy_image, 0, 255)
    return noisy_image
tasks = {
    "add_noise":add_noise,
    "random_rotate":random_rotate,
    "vertical_flip":vertical_flip,
    "horizontal_flip":horizontal_flip,
    "horizontal_vertical_flip":horizontal_vertical_flip,
    "horizontal_shearing":horizontal_shearing,
    "vertical_shearing":vertical_shearing,
    "horizontal_vertical_shearing":horizontal_vertical_shearing,
    "salt_and_pepper_noise":salt_and_pepper_noise,
    "contrasting": random_brightness_contrast,
    "grayscale": grayscale_3channels,
    "poisson_noise": poisson_noise
}

def data_augmentation(path):
    choices = list(tasks.keys())
    random_choice = random.choice(choices)
    img = cv2.imread(path)
    img = tasks[random_choice](img)
    return img



In [4]:
benign_files = list(filter(lambda x: x.startswith("ISIC"),os.listdir(os.path.join("new_training_data","benign"))))

In [5]:
malignant_files = list(filter(lambda x: x.startswith("ISIC"),os.listdir(os.path.join("new_training_data","malignant"))))

In [10]:
for _ in range(100000):
    file = random.choice(malignant_files)
    img = data_augmentation(os.path.join("new_training_data","malignant",file))
    filename = os.path.join("new_training_data","malignant",str(uuid.uuid4())+".jpg")
    cv2.imwrite(filename,img)

KeyboardInterrupt: 

In [ ]:
for _ in range(100000):
    file = random.choice(benign_files)
    img = data_augmentation(os.path.join("new_training_data","benign",file))
    filename = os.path.join("new_training_data","benign",str(uuid.uuid4())+".jpg")
    cv2.imwrite(filename,img)